# Modelado de topicos para la clusterizacion y predicciones de varietales

### Importamos las librerias y paquetes necesarios

Librerias

In [1]:
import pandas as pd
import numpy as np
import nltk

Paquetes especificos

In [2]:
from IPython.display import display_html
from nltk.corpus import stopwords

Para el caso que no tegamos la lista de stopWords, la podemos descargar con el siguiente comando

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/nico/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

********************************

### Seteamos algunas cosas y definimos funciones para mas adelante

Comenzamos con algunas cosas que vamos a necesitar mas adelante

In [4]:
# La creacion de la lista de stopsWords customizada se debe a que en cada topico existe un grupo de palabras que no 
# aportan informacion relevante para el analisis, en este caso, por ejemplo, la palabra wine, no tiene relevancia
# ya es un pablara que hace referencia al objeto de analisis direcamente sin destacar niguna propiedad del el
customStopWords = ['wine', 'flavors']

In [5]:
#Stop Words que incluyen por defecto la libreria NLTK
stopWords = stopwords.words('english') + customStopWords

In [7]:
# Esta funcion nos va a permitir mas adelante
# visualizar algunos DataFrames de manera simultanea
def mydisplay(dfs, names=[]):

    count = 0
    maxTables = 6
    
    if not names:
        names = [x for x in range(len(dfs))]

    html_str = ''
    html_th = ''
    html_td = ''
    
    for df, name in zip(dfs, names):
        if count <= (maxTables):
            html_th += (''.join(f'<th style="text-align:center">{name}</th>'))
            html_td += (''.join(f'<td style="vertical-align:top"> {df.to_html(index=False)}</td>'))
            count += 1
        else:
            html_str += f'<tr>{html_th}</tr><tr>{html_td}</tr>'
            html_th = f'<th style="text-align:center">{name}</th>'
            html_td = f'<td style="vertical-align:top"> {df.to_html(index=False)}</td>'
            count = 0
    
    
    if count != 0:
        html_str += f'<tr>{html_th}</tr><tr>{html_td}</tr>'
        
    
    html_str += f'<table>{html_str}</table>'
    html_str = html_str.replace('table','table style="display:inline"')
    display_html(html_str, raw=True)

***********

### Importamos el set de datos

Set de Datos

Los datos fueron tomados de sitio https://www.kaggle.com
Publicados en https://www.kaggle.com/zynicide/wine-reviews 


In [9]:
# df = pd.read_csv('../input/winemag-data-130k-v2.csv')
df = pd.read_csv('../data/winemag-data-130k-v2.csv', index_col = None)
df.drop(columns=['Unnamed: 0'], inplace = True)

### Realizamos las primeras transformaciones y comenzamos con el analisis exploratorio

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129971 entries, 0 to 129970
Data columns (total 13 columns):
country                  129908 non-null object
description              129971 non-null object
designation              92506 non-null object
points                   129971 non-null int64
price                    120975 non-null float64
province                 129908 non-null object
region_1                 108724 non-null object
region_2                 50511 non-null object
taster_name              103727 non-null object
taster_twitter_handle    98758 non-null object
title                    129971 non-null object
variety                  129970 non-null object
winery                   129971 non-null object
dtypes: float64(1), int64(1), object(11)
memory usage: 12.9+ MB


In [12]:
# Dado que hay solo una fila con la descripcion 
# nula, vamos a borrar ese dato
df.dropna(how='any', inplace=True)

In [13]:
#Pasamos algunos datos de texto a minuscula
df.description = df.description.apply(lambda x : x.lower())
df.variety = df.variety.apply(lambda x : x.lower())

In [14]:
#Sample data of all columns
df.sample(1)

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
114664,US,"this wine is forward and high-toned, with brig...",Two Messengers,87,30.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Maison L'Envoyé 2015 Two Messengers Pinot Noir...,pinot noir,Maison L'Envoyé


In [15]:
#Amount of uniques varieties
print('Se encuentrar ',df.variety.unique().size, ' variedades diferentes')
print('Hay vinos de ',df.country.unique().size, ' paises', end=' ')
print('y',df.winery.unique().size, ' bodegas diferentes')

Se encuentrar  183  variedades diferentes
Hay vinos de  1  paises y 2791  bodegas diferentes


Top 20 variatales mas revisados

In [17]:
pd.DataFrame(df.variety.value_counts()[:20]).reset_index(drop=False).rename(columns={'index':'varietal','variety':'recuento'})

,varietal,recuento
0,pinot noir,4788
1,chardonnay,2407
2,cabernet sauvignon,2372
3,red blend,1803
4,syrah,1678
5,zinfandel,1114
6,bordeaux-style red blend,1112
7,merlot,597
8,sauvignon blanc,574
9,rhône-style red blend,516


##### Revision de los nombres de las variedades de los vinos

In [28]:
#Aqui quitamos las stopsWords
desc = df[['description', 'variety']].copy()
desc.description =  desc.description.apply(lambda x : ' '.join([w for w in x.split() if w.lower() not in stopWords]))

In [30]:
print('Numero medio de palabras con las que se describen los varietales')
round(desc.description.apply(lambda x : len(x.split())).mean(), 2)

Numero medio de palabras con las que se describen los varietales


26.22

Vamos a evaluar un varietal, Pinot Noir. Vamos a tomar todas las descripciones y hacer un lista de las palabras que se usaron


-> Acordate que pasamos todo a minuscula

In [38]:
pinotNoir = pd.Series((' '.join([x for x in desc.groupby('variety').get_group('pinot noir').description]).split()))

In [40]:
print('Evaluacion de las descripciones del varital Pinot Noir')
print('Cantidad de palabras distintas usadas :', pinotNoir.unique().size)
print('Aparicion promedio de palabras : ', round(pinotNoir.value_counts().mean(), 2))
print('Lista ordenada de las palabras usadas (top 5):')
pinotNoir.value_counts()[:5]

Evaluacion de las descripciones del varital Pinot Noir
Cantidad de palabras distintas usadas : 13130
Aparicion promedio de palabras :  9.89
Lista ordenada de las palabras usadas (top 5):


fruit      1513
cherry     1465
black      1421
red         795
finish.     786
dtype: int64

#### Teniendo en cuanta lo anterior vamos a entrar en un aanalisi de palabras sobre como cada palabra describe a un varietal y cuan comun o especifica es cada palabra a la hora de describir un varietal. Recordemos que estas manopulaciones de texto tienen el obeto de reducir el ruido en el analisis de clusterin mas adelante

In [29]:
#Palabras econtradas en los varietales
df1 = pd.DataFrame(pd.Series(' '.join([w for w in desc.variety]).split()).value_counts()[:20]).reset_index(drop=False).rename(columns={'index':'words',0:'recuento'})

In [22]:
df1

,words,recuento
0,pinot,5173
1,noir,4801
2,blend,4274
3,red,3431
4,sauvignon,3006
5,cabernet,2749
6,chardonnay,2407
7,syrah,1678
8,bordeaux-style,1138
9,zinfandel,1114


In [23]:
#Varietales que contienen las palabras mas usadas
dfs = list()
for w in df1.words:
    word = w
    data = pd.DataFrame(pd.Series([x for x in cleanNan.variety if w in x]).value_counts()).reset_index(drop=False).rename(columns={'index':'word {}'.format(w),0:'recuento'})
    
    dfs.append((data,word))

In [27]:
#Varietals that contains the most common words. Aca la funcion de vsualizacion
mydisplay([d[0] for d in dfs], [w[1] for w in dfs])

pinot noir blend red sauvignon cabernet chardonnay 
 
 
 word pinot 
 recuento 
 
 
 
 
 pinot noir 
 4788 
 
 
 pinot gris 
 269 
 
 
 pinot blanc 
 57 
 
 
 pinot grigio 
 52 
 
 
 pinot-chardonnay 
 5 
 
 
 pinotage 
 2 
 
 
 pinot meunier 
 2 
 
 
 pinot noir-syrah 
 2 
 
 
 pinot blanc-pinot noir 
 1 
 
 
 pinot gris-gewürztraminer 
 1 
 
 
 chardonnay-pinot blanc 
 1 
 
 
 pinot auxerrois 
 1 
 
 
 
 
 
 word noir 
 recuento 
 
 
 
 
 pinot noir 
 4788 
 
 
 gamay noir 
 9 
 
 
 grenache noir 
 2 
 
 
 pinot noir-syrah 
 2 
 
 
 pinot blanc-pinot noir 
 1 
 
 
 baco noir 
 1 
 
 
 
 
 
 word blend 
 recuento 
 
 
 
 
 red blend 
 1803 
 
 
 bordeaux-style red blend 
 1112 
 
 
 rhône-style red blend 
 516 
 
 
 white blend 
 334 
 
 
 sparkling blend 
 299 
 
 
 rhône-style white blend 
 154 
 
 
 bordeaux-style white blend 
 26 
 
 
 cabernet blend 
 11 
 
 
 champagne blend 
 10 
 
 
 tempranillo blend 
 5 
 
 
 grenache blend 
 3 
 
 
 touriga nacional blend 
 1 
 
 
 
 
 
 word red 
 recuento 
 
 
 
 
 red blend 
 1803 
 
 
 bordeaux-style red blend 
 1112 
 
 
 rhône-style red blend 
 516 
 
 
 
 
 
 word sauvignon 
 recuento 
 
 
 
 
 cabernet sauvignon 
 2372 
 
 
 sauvignon blanc 
 574 
 
 
 cabernet sauvignon-syrah 
 38 
 
 
 sauvignon blanc-semillon 
 24 
 
 
 syrah-cabernet sauvignon 
 21 
 
 
 cabernet sauvignon-merlot 
 18 
 
 
 semillon-sauvignon blanc 
 16 
 
 
 merlot-cabernet sauvignon 
 9 
 
 
 cabernet sauvignon-cabernet franc 
 5 
 
 
 cabernet sauvignon-sangiovese 
 4 
 
 
 sangiovese-cabernet sauvignon 
 3 
 
 
 tempranillo-cabernet sauvignon 
 2 
 
 
 cabernet sauvignon-malbec 
 1 
 
 
 malbec-cabernet sauvignon 
 1 
 
 
 
 
 
 word cabernet 
 recuento 
 
 
 
 
 cabernet sauvignon 
 2372 
 
 
 cabernet franc 
 284 
 
 
 cabernet sauvignon-syrah 
 38 
 
 
 syrah-cabernet sauvignon 
 21 
 
 
 cabernet sauvignon-merlot 
 18 
 
 
 cabernet blend 
 11 
 
 
 merlot-cabernet sauvignon 
 9 
 
 
 cabernet franc-merlot 
 9 
 
 
 merlot-cabernet 
 5 
 
 
 cabernet sauvignon-cabernet franc 
 5 
 
 
 merlot-cabernet franc 
 4 
 
 
 sangiovese cabernet 
 4 
 
 
 cabernet sauvignon-sangiovese 
 4 
 
 
 cabernet-syrah 
 3 
 
 
 sangiovese-cabernet sauvignon 
 3 
 
 
 syrah-cabernet 
 3 
 
 
 cabernet merlot 
 3 
 
 
 tempranillo-cabernet sauvignon 
 2 
 
 
 malbec-cabernet sauvignon 
 1 
 
 
 cabernet sauvignon-malbec 
 1 
 
 
 
 
 
 word chardonnay 
 recuento 
 
 
 
 
 chardonnay 
 2407 
 
 
 pinot-chardonnay 
 5 
 
 
 chardonnay-viognier 
 3 
 
 
 chardonnay-semillon 
 3 
 
 
 viognier-chardonnay 
 2 
 
 
 semillon-chardonnay 
 1 
 
 
 chardonnay-pinot blanc 
 1 
 
 
 syrah bordeaux-style zinfandel blanc rhône-style merlot white riesling 
 
 
 word syrah 
 recuento 
 
 
 
 
 syrah 
 1678 
 
 
 cabernet sauvignon-syrah 
 38 
 
 
 syrah-grenache 
 22 
 
 
 syrah-cabernet sauvignon 
 21 
 
 
 grenache-syrah 
 17 
 
 
 syrah-mourvèdre 
 7 
 
 
 syrah-petite sirah 
 7 
 
 
 sangiovese-syrah 
 3 
 
 
 cabernet-syrah 
 3 
 
 
 syrah-viognier 
 3 
 
 
 syrah-cabernet 
 3 
 
 
 syrah-tempranillo 
 2 
 
 
 pinot noir-syrah 
 2 
 
 
 tempranillo-syrah 
 1 
 
 
 syrah-merlot 
 1 
 
 
 syrah-petit verdot 
 1 
 
 
 syrah-grenache-viognier 
 1 
 
 
 mourvèdre-syrah 
 1 
 
 
 tannat-syrah 
 1 
 
 
 
 
 
 word bordeaux-style 
 recuento 
 
 
 
 
 bordeaux-style red blend 
 1112 
 
 
 bordeaux-style white blend 
 26 
 
 
 
 
 
 word zinfandel 
 recuento 
 
 
 
 
 zinfandel 
 1114 
 
 
 
 
 
 word blanc 
 recuento 
 
 
 
 
 sauvignon blanc 
 574 
 
 
 chenin blanc 
 57 
 
 
 pinot blanc 
 57 
 
 
 grenache blanc 
 51 
 
 
 sauvignon blanc-semillon 
 24 
 
 
 semillon-sauvignon blanc 
 16 
 
 
 fumé blanc 
 14 
 
 
 vidal blanc 
 5 
 
 
 ugni blanc 
 3 
 
 
 muscat blanc 
 2 
 
 
 muscat blanc à petits grains 
 1 
 
 
 chardonnay-pinot blanc 
 1 
 
 
 garnacha blanca 
 1 
 
 
 pinot blanc-pinot noir 
 1 
 
 
 
 
 
 word rhône-style 
 recuento 
 
 
 
 
 rhône-style red blend 
 516 
 
 
 rhône-style white blend 
 154 
 
 
 
 
 
 word 

In [41]:
Esto por ahora...

SyntaxError: invalid syntax (<ipython-input-41-fd42a4b9ba7c>, line 1)